In [ ]:
from google.cloud import bigquery

In [ ]:
from shapely import geometry, wkt

In [ ]:
import geopandas as gpd

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import os

In [ ]:
root = os.path.abspath(os.path.join(os.getcwd(),'..'))

In [ ]:
client = bigquery.Client()

In [ ]:
Q = f"""
    SELECT *
    FROM `oxeo-main.wave2web.tracked-reservoirs`
"""

In [ ]:
df = client.query(Q).result().to_dataframe()

In [ ]:
df['geometry'] = df['upstream_geom'].apply(wkt.loads)

In [ ]:
gdf= gpd.GeoDataFrame(df, geometry='geometry')

In [ ]:
gdf.boundary.plot()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(16,16))
gdf.boundary.plot(ax=ax)
for idx, row in gdf.iterrows():
    ax.text(row['geometry'].centroid.x, row['geometry'].centroid.y, row['name'])

In [ ]:
gdf = gdf.drop(columns=['uuid','upstream_geom']).rename(columns={'name':'DAM_NAME'})

In [ ]:
gdf.to_file(os.path.join(root,'data','basins_all.geojson'))

In [ ]:
gdf['DAM_NAME'].values

In [ ]:
client = bigquery.Client()

        query_keys = site_mapper.keys()

        sites_query = '("' + '","'.join(query_keys) + '")'

        # construct query
        Q = f"""
            SELECT {', '.join(variable_keys)}
            FROM `{bq_address}`
            WHERE
              {site_col} in {sites_query}
              AND {datetime_col} <= "{end_datetime.isoformat()[0:10]}"
              AND {datetime_col} >= "{start_datetime.isoformat()[0:10]}"
        """

        # execute query
        df = client.query(Q).result().to_dataframe()